In [ ]:
import piplite

await piplite.install('ipytone')

In [ ]:
import ipytone
import ipywidgets as widgets

#### Synths, effects and audio routing

In [ ]:
channel1 = ipytone.Channel(pan=-0.5).to_destination()
filtr = ipytone.Filter(frequency=100)
synth = ipytone.MonoSynth(volume=-7).chain(filtr, channel1)

channel2 = ipytone.Channel(pan=0.2, channel_count=2).to_destination()
delay = ipytone.PingPongDelay(delay_time="16n", feedback=0.2)
perc_synth = ipytone.MembraneSynth(volume=-10).chain(delay, channel2)

In [ ]:
lfo = ipytone.LFO(frequency="4m", min=100, max=10_000)
lfo.connect(filtr.frequency).start()

lfo2 = ipytone.LFO(frequency="8n", min=-200, max=200, type="triangle")
lfo2.connect(perc_synth.detune).start()

perc_synth.pitch_decay = 0.02
delay.wet.value = 0.1
lfo2.amplitude.value = 0.6

#### Music loop

In [ ]:
def clb(time, note):
    synth.trigger_attack_release(note, 0.2, time=time)
    perc_synth.trigger_attack_release(note, 0.05, time=time)

sequence = ipytone.Sequence(
    callback=clb,
    events=["A0", "A1", "A0", None, "F#2", "G2", "G#2", "A2"],
    subdivision="16n",
)

#### Play it!

In [ ]:
ipytone.transport.start()
sequence.start()

Pause it right now if you forgot to adjust the volume of the speakers!!

In [ ]:
ipytone.transport.pause()

Ok now? Let's resume and have some fun...

In [ ]:
ipytone.transport.start()

In [ ]:
import asyncio

async def melody():
    """8x2 measures melody"""
    m2 = ipytone.transport.bpm.value / 60 * 2

    # 1 ---
    sequence.events = ["A1", "A2", "A0", None, "E4", "A3", "E4", "A4"]
    await asyncio.sleep(m2)
    # 2 ---
    sequence.events = ["A1", "A2", "A0", None, "E4", "A3", "E4", "B4"]
    await asyncio.sleep(m2)
    # 3 ---
    sequence.events = ["A1", "A2", "A0", None, "E4", "A3", "E4", "C4"]
    await asyncio.sleep(m2)
    # 4 ---
    sequence.events = ["A1", "A2", None, "A0", "E5", None, "E4", "A4"]
    await asyncio.sleep(m2)
    # 5 ---
    sequence.events = ["A1", None, None, "A0", "E5", None, None, None]
    await asyncio.sleep(m2)
    # 6 ---
    synth.portamento = 0.08
    filtr.q.ramp_to(6, "4m")
    lfo2.amplitude.ramp_to(1, "4m")
    await asyncio.sleep(m2)
    # 7 ---
    sequence.events = ["A1", "A2", None, "A0", "E5", None, "G5", "A4"]
    await asyncio.sleep(m2)
    # 8 ---
    delay.wet.ramp_to(0.8, "2m")
    synth.portamento = 0.1
    await asyncio.sleep(m2)
    # 1 ---
    synth.portamento = 0
    filtr.q.ramp_to(1, "4n")
    delay.wet.ramp_to(0.1, "4n")
    lfo.amplitude.ramp_to(0.1, "4n")
    sequence.events = ["A0", "A1", "A0", None, "F#2", "G2", "G#2", "A2"]


In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(melody());

#### Mixer

In [ ]:
def create_channel(node):
    # panner
    pan = widgets.FloatSlider(
        value=node.pan.value, min=-1, max=1,
        layout=widgets.Layout(width="200px")
    )
    widgets.jslink((pan, "value"), (node.pan, "value"))
    
    # solo / mute buttons
    solo = widgets.ToggleButton(value=False, description="Solo")
    mute = widgets.ToggleButton(value=False, description="Mute")

    def node_solo(change):
        node.solo = change['new']
        
    def node_mute(change):
        node.mute = change["new"]
        
    solo.observe(node_solo, names='value')
    mute.observe(node_mute, names='value')
    
    # fader
    fader = widgets.FloatSlider(
        value=0, min=-30, max=4, orientation="vertical"
    )
    widgets.jslink((fader, "value"), (node.volume, "value"))

    # L/R VU meters
    vu_left = widgets.FloatProgress(
        min=0, max=0.4, orientation="vertical"
    )
    vu_right = widgets.FloatProgress(
        min=0, max=0.4, orientation="vertical"
    )
    split = ipytone.Split()
    node.connect(split)
    meter_left = ipytone.Meter(normal_range=True)
    split.connect(meter_left, 0, 0)
    meter_left.schedule_jsdlink((vu_left, "value"), transport=True)
    meter_right = ipytone.Meter(normal_range=True)
    split.connect(meter_right, 1, 0)
    meter_right.schedule_jsdlink((vu_right, "value"), transport=True)
    
    # layout
    fader_vus = widgets.HBox([fader, vu_left, vu_right])
    return widgets.VBox([pan, solo, mute, fader_vus])


widgets.HBox([create_channel(channel1), create_channel(channel2)])

#### Enough!

In [ ]:
sequence.stop()

In [ ]:
ipytone.transport.stop()

#### Dispose (free audio ressource)

In [ ]:
synth.dispose()
filtr.dispose()
lfo.dispose()
lfo2.dispose()
channel1.dispose()
perc_synth.dispose()
delay.dispose()
channel2.dispose()

sequence.dispose()